# Extracting the JAAD images from videos and showing the crossing prediction

In [ ]:
import numpy as np

from torch import from_numpy
from torch import cuda
from torch import no_grad
from torch import optim

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvasAgg

%matplotlib inline

from Code.GNN import *
from Code.SkeletonsDataset import *
from Code.ModelTrainEvaluate import *

import cv2
from PIL import Image

## Loading the skeletons dataset

In [2]:
datasetName = 'JAAD'
subset = 'test'
poseEstimator = 'AlphaPose'
numberOfClasses = 2
info = 87
data_augm = '' #'dataAugmentation4'

net = 'TGCN'
dropout = 0.5
num_epochs = 1000
batch_size = 10000

data_augmentation_name = '' if data_augm == None or data_augm == '' else '_' + data_augm
numberOfJoints = 25 if poseEstimator == 'OpenPose' else 18

dataset = SkeletonsDataset('Data/' + datasetName + '/' + subset + '_jaad_' + poseEstimator + '.csv',
                           normalization='minmax', target='cross', info=info,
                           remove_undetected=True, numberOfJoints=numberOfJoints)

In [3]:
videos_list = dataset.loadedData['video'].unique().tolist()

videos_list

['video_0005',
 'video_0016',
 'video_0017',
 'video_0028',
 'video_0042',
 'video_0045',
 'video_0046',
 'video_0048',
 'video_0053',
 'video_0055',
 'video_0059',
 'video_0071',
 'video_0076',
 'video_0084',
 'video_0087',
 'video_0090',
 'video_0092',
 'video_0093',
 'video_0096',
 'video_0097',
 'video_0100',
 'video_0101',
 'video_0103',
 'video_0104',
 'video_0105',
 'video_0106',
 'video_0107',
 'video_0110',
 'video_0113',
 'video_0115',
 'video_0116',
 'video_0117',
 'video_0118',
 'video_0124',
 'video_0125',
 'video_0128',
 'video_0135',
 'video_0141',
 'video_0144',
 'video_0148',
 'video_0150',
 'video_0151',
 'video_0152',
 'video_0155',
 'video_0162',
 'video_0163',
 'video_0164',
 'video_0165',
 'video_0173',
 'video_0177',
 'video_0178',
 'video_0179',
 'video_0183',
 'video_0187',
 'video_0197',
 'video_0201',
 'video_0203',
 'video_0206',
 'video_0211',
 'video_0212',
 'video_0213',
 'video_0216',
 'video_0221',
 'video_0222',
 'video_0223',
 'video_0224',
 'video_02

In [4]:
dataset.loadedData[['video','frame','skeleton','cross']]

,video,frame,skeleton,cross
0,video_0005,0,"[[998.39892578125, 694.7347412109375, 0.937213...",not-crossing
1,video_0005,1,"[[998.4149169921875, 694.6920166015625, 0.9358...",not-crossing
2,video_0005,2,"[[998.3433227539062, 696.0174560546875, 0.9455...",not-crossing
3,video_0005,3,"[[997.6259765625, 696.2772216796875, 0.9139388...",not-crossing
4,video_0005,4,"[[996.3665161132812, 697.2818603515625, 0.9003...",not-crossing
...,...,...,...,...
52961,video_0344,85,"[[1767.3865966796875, 764.985107421875, 0.9258...",not-crossing
52962,video_0344,86,"[[1785.76220703125, 762.012451171875, 0.931533...",not-crossing
52963,video_0344,87,"[[1807.765625, 760.1240234375, 0.8868738412857...",not-crossing
52964,video_0344,88,"[[1858.822021484375, 766.7747802734375, 0.9220...",not-crossing


In [5]:
# First element of the dataset:
t0 = dataset[0]

# Node features:
t1 = t0.x_temporal[0]

# Number of nodes:
numberOfNodes = t1.shape[0]

# Number of dimensions of each node features:
embed_dim = t1.shape[1]

print('Number of nodes per skeleton:', numberOfNodes)
print('Number of features per node:', embed_dim)

Number of nodes per skeleton: 18
Number of features per node: 3


## Loading the trained model

In [6]:
modelName = net + '_' + poseEstimator + '_info=' + str(info) + '_dropout=' + str(dropout) + data_augmentation_name
modelName = modelName + '_epoch=' + str(num_epochs)

print('Loading model:', modelName)

device = torch.device('cuda')

model = SpatialTemporalGNN(embed_dim, numberOfClasses, numberOfNodes, net=net,
                           filterSize=embed_dim, dropout=dropout, batchSize=batch_size).to(device)

model.load_state_dict(torch.load('exportedModels/' + datasetName + '/' + modelName))

Loading model: TGCN_AlphaPose_info=87_dropout=0.5_epoch=1000


<All keys matched successfully>

## Making the crossing/not-crossing prediction

In [7]:
loader = DataLoader(dataset, batch_size=batch_size)

predictions, groundtruth = predict(model, loader, device)

## Loading the video and exporting the result as a GIF

In [8]:
import os

os.makedirs("Videos_results/" + datasetName + '/' + poseEstimator + "/" + subset + "/" + modelName, exist_ok=True)

quantity = 10

gen_clips = os.listdir("Data/" + datasetName + "-videos/")

videos_list_frames = dataset.loadedData['video'].tolist()

exported = 0

for video_id in videos_list[0:10]:
    
    print('Starting processing of video', video_id)

    video = cv2.VideoCapture("Data/" + datasetName + "-videos/" + video_id + ".mp4")


    # First column in the dataset where the video starts:
    video_first_dataset_row = videos_list_frames.index(video_id)
    
    frames = dataset.loadedData[dataset.loadedData['video'] == video_id].sort_values(by=['frame', 'ped_id'], ascending=True)
    frames = frames['frame'].tolist()
    frames_rev = frames[::-1]

    video_outputs = []

    frame_i = 0
    ret = True
    while ret:

        ret, frame = video.read()

        if ret:
            
            frame = frame[...,::-1]
            
            if frame_i in frames:
            
                frame_first_index = frames.index(frame_i)
                frame_last_index = len(frames_rev) - frames_rev.index(frame_i) - 1

                frame_prediction = predictions[video_first_dataset_row + frame_first_index:video_first_dataset_row + frame_last_index + 1]
                frame_groundtruth = groundtruth[video_first_dataset_row + frame_first_index:video_first_dataset_row + frame_last_index + 1]

                frame_prediction = ["Crossing" if f else "Not-crossing" for f in frame_prediction]
                frame_groundtruth = ["Crossing" if f else "Not-crossing" for f in frame_groundtruth]

                #im_title = "Prediction: " + frame_prediction + "\nGroundtruth: " + frame_groundtruth
                im_title = "Video: " + video_id
                im_title = im_title + " - Frame: " + str(frame_i)
                im_title = im_title + "\nPose estimator: " + poseEstimator #if frame_i == 0 else ""
                im_title = im_title + "\nClassifier: " + net + '_info=' + str(info) + '_dropout=' + str(dropout) + data_augmentation_name + '_epoch=' + str(num_epochs)


                fig = dataset.showSkeleton(videoNum=video_id, frameNum=frame_i, showLegend=False, frameImage=frame,
                                           normalizedSkeletons=False, title=im_title, show=False,
                                           predictions=frame_prediction, groundtruths=frame_groundtruth)


                canvas = FigureCanvasAgg(fig)
                canvas.draw()
                frame_result = np.asarray(canvas.buffer_rgba()).astype(np.uint8)

                frame_result = Image.fromarray(frame_result)

                video_outputs.append(frame_result)

                canvas.get_renderer().clear()
                plt.close(fig)

        frame_i = frame_i + 1
                

    
    # Export the prediction result as a GIF:
    if len(video_outputs) > 0:
        video_outputs[0].save("Videos_results/" + datasetName + '/' + poseEstimator + "/" + subset + "/" + modelName + "/" + video_id + ".gif", 
                              save_all=True, append_images=video_outputs[1:], duration=30, loop=0, disposal=2)
    
    exported = exported + 1
    
    print('Exported video:', video_id, ' - Clip', str(exported) + '/' + str(quantity), '\n')
    
    video.release()
    
    if exported == quantity:
        break

Starting processing of video video_0005
Exported video: video_0005  - Clip 1/10 

Starting processing of video video_0016
Exported video: video_0016  - Clip 2/10 

Starting processing of video video_0017
Exported video: video_0017  - Clip 3/10 

Starting processing of video video_0028
Exported video: video_0028  - Clip 4/10 

Starting processing of video video_0042
Exported video: video_0042  - Clip 5/10 

Starting processing of video video_0045
Exported video: video_0045  - Clip 6/10 

Starting processing of video video_0046
Exported video: video_0046  - Clip 7/10 

Starting processing of video video_0048
Exported video: video_0048  - Clip 8/10 

Starting processing of video video_0053
Exported video: video_0053  - Clip 9/10 

Starting processing of video video_0055
Exported video: video_0055  - Clip 10/10 

